In [10]:
import pandas as pd
dataset = pd.read_csv('IMDB Dataset.csv')
m = dataset.review.as_matrix()

/home/mike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
labels = (dataset.sentiment=='positive').as_matrix().astype(int)

/home/mike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [14]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8)
processed_features = vectorizer.fit_transform(m.tolist()).toarray()

In [20]:
import numpy as np
np.where(processed_features[10])

(array([  46,   79,  120,  126,  135,  148,  154,  166,  199,  237,  280,
         296,  303,  309,  366,  589,  730,  735,  833,  838,  842,  850,
         875,  906,  913, 1092, 1108, 1142, 1144, 1159, 1160, 1184, 1195,
        1315, 1328, 1447, 1487, 1526, 1541, 1552, 1632, 1682, 1693, 1746,
        1962, 2002, 2006, 2186, 2190, 2201, 2214, 2276, 2343, 2387, 2409,
        2420, 2426, 2475]),)

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB

#text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier = BernoulliNB()#n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [36]:
predictions = text_classifier.predict(X_test)

In [37]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[4132  903]
 [ 747 4218]]
              precision    recall  f1-score   support

           0       0.85      0.82      0.83      5035
           1       0.82      0.85      0.84      4965

   micro avg       0.83      0.83      0.83     10000
   macro avg       0.84      0.84      0.83     10000
weighted avg       0.84      0.83      0.83     10000

0.835
